In [14]:
# I chose the Exponential Smoothing (Holt-Winters) model

# assignment1.py  —— 只暴露三个变量：model, modelFit, pred
import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# -------- 1) 读取本地训练/测试数据（相对路径，适配 GitHub Actions） --------
import os

# 默认先尝试本地路径（GitHub autograder 用）
TRAIN_PATH = "assignment_data_train.csv"
TEST_PATH  = "assignment_data_test.csv"

# 如果本地不存在，就用 GitHub 原始链接（Colab 调试用）
if not os.path.exists(TRAIN_PATH):
    TRAIN_PATH = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment1/main/assignment_data_train.csv"
    TEST_PATH  = "https://raw.githubusercontent.com/dustywhite7/econ8310-assignment1/main/assignment_data_test.csv"

train = pd.read_csv(TRAIN_PATH, parse_dates=["Timestamp"])
test  = pd.read_csv(TEST_PATH,  parse_dates=["Timestamp"])


# 时间索引 + 小时频率；目标强制为数值并双向插值，避免初始化报错
train = train.sort_values("Timestamp").set_index("Timestamp").asfreq("H")
test  = test.sort_values("Timestamp").set_index("Timestamp").asfreq("H")

y_train = pd.to_numeric(train["trips"], errors="coerce").interpolate(
    limit_direction="both"
)

# -------- 2) 定义模型（允许的类型：Holt-Winters / ExponentialSmoothing）--------
# 小时数据 → 24 小时季节；用加法趋势 + 加法季节；初始化用 estimated 以兼容缺失
model = ExponentialSmoothing(
    y_train,
    trend="add",
    seasonal="add",
    seasonal_periods=24,
    initialization_method="estimated",
)

# -------- 3) 拟合并预测 --------
# optimized=True 让 statsmodels 自动寻参；多数环境不需要 use_brute
modelFit = model.fit(optimized=True)

h = len(test)  # 期待 744
forecasts = modelFit.forecast(steps=h)

# -------- 4) 产出一维数组/列表（通过 testValidPred）--------
pred = np.asarray(forecasts, dtype=float).ravel().tolist()


/tmp/ipython-input-655176934.py:25: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  train = train.sort_values("Timestamp").set_index("Timestamp").asfreq("H")
/tmp/ipython-input-655176934.py:26: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  test  = test.sort_values("Timestamp").set_index("Timestamp").asfreq("H")
